In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import pearsonr
import pandas as pd

import sys
sys.path.append('../../src/')
from coal_cov import *
from seg_sites_covar import CorrSegSites
from plot_utils import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['figure.facecolor'] = "w"
plt.rcParams['figure.autolayout'] = True
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mticker

import os
main_figdir = '../../plots/two_locus_stats/'
supp_figdir = '../../plots/supp_figs/two_locus_stats/'
os.makedirs(main_figdir, exist_ok=True)
os.makedirs(supp_figdir, exist_ok=True)

### Simulation under a model of exponential growth 

In [3]:
# NOTE : growth always starts ~ 1000 Generations in the past ... 
np.random.seed(42)

tas = [0, 100, 500]
rs = [1e-3, 1e-2, 5e-2]
rec_rate = 1e-3


total_sims = []
for t in tqdm(tas):
    for r in rs:
        cur_sim = TwoLocusSerialGrowth(ta=t, r=r, T=500, reps=50000, rec_rate=rec_rate)
        ts_reps = cur_sim._simulate()
        cur_sim._two_locus_branch_length(ts_reps)
        # Calculating the marginal variance and means
        mu_LA= np.mean(cur_sim.pair_branch_length[:,0])
        var_LA = np.var(cur_sim.pair_branch_length[:,0])
        cov_LALB = np.cov(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0,1]
        corr_LALB = pearsonr(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0]
        total_sims.append([t,r, mu_LA, var_LA, cov_LALB, corr_LALB])
            
total_sims = np.vstack(total_sims)

100%|██████████| 3/3 [02:29<00:00, 49.86s/it]


In [4]:
growth_df = pd.DataFrame(total_sims)
growth_df.columns = ['$t_a$', '$r$','$\mathbb{E}[L]$','$Var(L)$','$Cov(L_A,L_B)$','$Corr(L_A, L_B)$']
growth_df.to_csv('../../results/two_loci/growth_model.csv', index=False)
growth_df

,$t_a$,$r$,$\mathbb{E}[L]$,$Var(L)$,"$Cov(L_A,L_B)$","$Corr(L_A, L_B)$"
0,0.0,0.001,6081.599994,4.689447e+06,766403.217594,0.163025
1,0.0,0.010,1943.183487,6.651699e+04,21191.205708,0.319441
2,0.0,0.050,1253.428771,2.629916e+03,983.265102,0.377029
3,100.0,0.001,5989.853215,4.607396e+06,779915.209659,0.169252
4,100.0,0.010,1843.912760,6.565337e+04,21294.679012,0.326416
5,100.0,0.050,1153.554370,2.591525e+03,1067.611787,0.412144
6,500.0,0.001,5686.418290,4.151856e+06,616245.689134,0.148448
7,500.0,0.010,1452.420484,5.865578e+04,24645.697444,0.419187
8,500.0,0.050,753.529799,2.497120e+03,1416.145306,0.565800


### Simulation under a bottleneck model

In [5]:
np.random.seed(42)

tas = [0, 100, 500]
rs = [1e-1, 1e-2, 1e-3]
rec_rate = 1e-3

total_sims_bot = []
for t in tqdm(tas):
    for r in rs:
        cur_sim = TwoLocusSerialBottleneck(Ne=1e4, ta=t, Tstart=50, Tend=500, Nbot=r*1e4, reps=50000, rec_rate=rec_rate)
        ts_reps = cur_sim._simulate()
        cur_sim._two_locus_branch_length(ts_reps)
        # Calculating the marginal variance and means
        mu_LA= np.mean(cur_sim.pair_branch_length[:,0])
        var_LA = np.var(cur_sim.pair_branch_length[:,0])
        cov_LALB = np.cov(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0,1]
        corrLALB = pearsonr(cur_sim.pair_branch_length[:,0], cur_sim.pair_branch_length[:,1])[0]
        total_sims_bot.append([t, r, mu_LA, var_LA, cov_LALB, corrLALB])
        
        
total_sims_bot = np.vstack(total_sims_bot)

100%|██████████| 3/3 [03:35<00:00, 71.89s/it]


In [6]:
bottleneck_df = pd.DataFrame(total_sims_bot)
bottleneck_df.columns = ['$t_a$', '$\phi$','$\mathbb{E}[L]$','$Var(L)$','$Cov(L_A,L_B)$','$Corr(L_A,L_B)$']
bottleneck_df.to_csv('../../results/two_loci/bottleneck.csv', index=False)
bottleneck_df

,$t_a$,$\phi$,$\mathbb{E}[L]$,$Var(L)$,"$Cov(L_A,L_B)$","$Corr(L_A,L_B)$"
0,0.0,0.100,33038.592683,1.552771e+09,1.882539e+08,0.122429
1,0.0,0.010,4690.353181,3.308344e+08,1.101634e+08,0.331987
2,0.0,0.001,139.793902,1.605542e+03,1.466509e+03,0.913137
3,100.0,0.100,33614.277155,1.552034e+09,1.812123e+08,0.116122
4,100.0,0.010,5787.370584,3.936224e+08,1.332749e+08,0.336735
5,100.0,0.001,140.117020,1.627716e+03,1.567128e+03,0.961984
6,500.0,0.100,40552.255278,1.614915e+09,3.581951e+07,0.022487
7,500.0,0.010,40266.147401,1.549269e+09,3.297263e+07,0.021029
8,500.0,0.001,40741.667095,1.599062e+09,3.402588e+07,0.021215
